In [1]:
 ! pip -q install pinecone-client huggingface_hub langchain langchain_community google.generativeai pypdf

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pinecone import Pinecone,ServerlessSpec
from langchain_community.llms import HuggingFaceHub
from langchain_community.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone as PineconeClient
from langchain.vectorstores import Pinecone
from dotenv import load_dotenv
import os
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI

In [4]:
huggingfacehub_api_token=os.getenv("huggingfacehub_api_token")

In [5]:
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
PINECONE_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]

In [6]:
# Init
pinecone = PineconeClient(api_key=PINECONE_API_KEY,
                         environment=PINECONE_ENVIRONMENT)
index_name=PINECONE_INDEX_NAME

# embeddings = CohereEmbeddings(model="multilingual-22-12")
vectorstore = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [7]:
loader = PyPDFDirectoryLoader("D:\Gita_GPT\data")
data = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)

In [9]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
embeddings = download_hugging_face_embeddings()

In [10]:
len(text_chunks)

In [12]:
docsearch=Pinecone.from_documents(text_chunks,embeddings,index_name=index_name)

In [13]:
docsearch=Pinecone.from_existing_index(index_name, embeddings)

In [15]:
prompt_template="""
Use the following pieces of information to answer the user's devote question.
dont write in the context, from the context
and provide the complete sentences of answer. 

Context: {context}
Question: {question}


Answer:
"""

In [16]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [18]:
llm = HuggingFaceHub(repo_id='mistralai/Mistral-7B-Instruct-v0.2',model_kwargs={"temperature":0.1}, huggingfacehub_api_token=huggingfacehub_api_token)

In [19]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [20]:
def answer_format(text):
    start_index = text.find("Answer:")
    if start_index == -1:
        return None
    return text[start_index:].strip()


In [21]:
while True:
  user_input=input(f"Question: ")

  result=qa({"query": user_input})
  result_text = result["result"]
  answer = answer_format(result_text)
  print(answer)
  print()
  print("------------------------------------------------------------------------------")

Question:  what is karma ?


C:\Users\mohan\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer:
Karma refers to the activities or actions that are carried out under the control and purview of eternal time. These activities, which have been ongoing from time immemorial, result in suffering or enjoyment for the living entity. For instance, a businessman who works hard and amasses a great bank balance is an enjoyer, but if he loses all his money, he becomes a sufferer. Similarly, every field of life brings results of our work, and this is called

------------------------------------------------------------------------------


Question:  which is the truth of universe ?


Answer:
The truth of the universe is that it is illuminated by the Supreme Brahman, and this great effulgence belongs to Lord Krishna. The proposal that matter alone is the source of life is less intelligent. The entire cosmic manifestation of the universe is developed due to the presence of the Supersoul, Viṣṇu. Spirit and matter, which combine together to manifest this gigantic universal form, are originally two energies of the Lord. The living entity, a

------------------------------------------------------------------------------


...

Question:  tell about the kali ?


Answer:
Kali-yuga is the current age of the cosmic time cycle, which according to the ancient Vedic treatise on astronomy, the Sürya-siddhånta, began in 3102 BCE. This age is known as the age of quarrel and hypocrisy, and virtuous qualities are reduced to 25%. During this age, sensual pleasure (Kåma) is prevalent, and the yogic

------------------------------------------------------------------------------


Question:  how to believe in god ?


Answer:
To believe in God, one must have faith in something that is productive of good works. However, in material conditional life, no works of material nature are completely purified. They are mixed and not in pure goodness. Pure goodness is transcendental, and in purified goodness, one can understand the real nature of the Supreme Personality of Godhead. As long as one's faith is not completely in purified goodness, it is subject to contamination by any of the modes

------------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user